### Imports

In [1]:
import pandas as pd
import requests

### Setup function to pull statistics from nba.com

In [2]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [3]:
seasons = ['2012-13',
          '2013-14',
          '2014-15',
          '2015-16',
          '2016-17',
          '2017-18',
          '2018-19',
          '2019-20',
          '2020-21',
          '2021-22']

In [5]:
def pull_stats(seasons, datefrom, measuretype, segment, numcol, columns):
    dfs = []
    for season, date in zip(seasons, datefrom):
        url = "https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom={}&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType={}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={}&SeasonSegment={}&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision="
        res = requests.get(url=url.format(date, measuretype, season, segment), headers=headers)
        res.raise_for_status()
        data = res.json()['resultSets'][0]['rowSet']
        extract = [data[i][0:numcol+1] for i in range(len(data))]
        df = pd.DataFrame(extract, columns=columns)
        df['Season'] = season
        dfs.append(df)
    return pd.concat(dfs, sort=False)

### Pull four factors team statistics + Save as a csv

In [6]:
### Four Factors parameters, all games pre all-star break

datefrom = [''] * 10
columns_ff = ["TeamID","Team","GP","W","L","WIN%","MIN","EFG%","FTA RATE","TOV%","OREB%","OPP EFG%","OPP FTA RATE","OPP TOV%","OPP OREB%"]

In [7]:
ff_stats = pull_stats(seasons, datefrom, 'Four+Factors', 'Pre+All-Star', 14, columns_ff)

In [8]:
ff_stats

,TeamID,Team,GP,W,L,WIN%,MIN,EFG%,FTA RATE,TOV%,OREB%,OPP EFG%,OPP FTA RATE,OPP TOV%,OPP OREB%,Season
0,1610612737,Atlanta Hawks,51,29,22,0.569,2478.0,0.516,0.241,0.161,0.265,0.491,0.239,0.162,0.299,2012-13
1,1610612738,Boston Celtics,52,28,24,0.538,2566.0,0.492,0.263,0.153,0.255,0.482,0.296,0.169,0.308,2012-13
2,1610612751,Brooklyn Nets,53,31,22,0.585,2574.0,0.489,0.306,0.162,0.338,0.502,0.235,0.149,0.303,2012-13
3,1610612766,Charlotte Bobcats,52,12,40,0.231,2521.0,0.455,0.316,0.151,0.311,0.516,0.261,0.149,0.328,2012-13
4,1610612741,Chicago Bulls,52,30,22,0.577,2516.0,0.468,0.271,0.162,0.344,0.467,0.268,0.153,0.312,2012-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,1610612758,Sacramento Kings,60,22,38,0.367,2895.0,0.522,0.252,0.141,0.273,0.543,0.226,0.131,0.290,2021-22
26,1610612759,San Antonio Spurs,59,23,36,0.390,2847.0,0.526,0.200,0.125,0.278,0.528,0.217,0.138,0.293,2021-22
27,1610612761,Toronto Raptors,57,32,25,0.561,2761.0,0.511,0.229,0.131,0.324,0.534,0.254,0.166,0.293,2021-22
28,1610612762,Utah Jazz,58,36,22,0.621,2784.0,0.557,0.265,0.145,0.290,0.516,0.209,0.124,0.264,2021-22


In [9]:
ff_stats.to_csv("four_factors.csv", index=False)

### Pull last ~15 games before AS break + Save as a csv

In [4]:
as_break_dates = ['01-13-2013',
                  '01-12-2014',
                  '01-11-2015',
                  '01-10-2016',
                  '01-15-2017',
                  '01-14-2018',
                  '01-13-2019',
                  '01-12-2020',
                  '02-03-2021',
                  '01-16-2022']

In [10]:
### Last 15 games parameters, ~15 games leading into all-star break

columns_last15 = ["TeamID","Team","GP","W","L","WIN%","MIN","EOFFRTG","OFFRTG","EDEFRTG","DEFRTG","ENETRTG","NETRTG"]

In [11]:
last_15 = pull_stats(seasons, as_break_dates, 'Advanced', 'Pre+All-Star', 12, columns_last15)

In [12]:
last_15

,TeamID,Team,GP,W,L,WIN%,MIN,EOFFRTG,OFFRTG,EDEFRTG,DEFRTG,ENETRTG,NETRTG,Season
0,1610612737,Atlanta Hawks,15,8,7,0.533,730.0,103.1,103.8,102.1,103.8,1.0,0.0,2012-13
1,1610612738,Boston Celtics,16,9,7,0.563,808.0,98.6,99.8,95.9,97.9,2.7,1.9,2012-13
2,1610612751,Brooklyn Nets,17,10,7,0.588,821.0,102.1,103.1,104.4,105.9,-2.3,-2.7,2012-13
3,1610612766,Charlotte Bobcats,16,3,13,0.188,768.0,95.5,97.6,107.9,109.5,-12.4,-11.8,2012-13
4,1610612741,Chicago Bulls,17,10,7,0.588,831.0,100.5,102.8,98.4,99.9,2.1,2.9,2012-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,1610612758,Sacramento Kings,15,4,11,0.267,720.0,108.1,110.3,117.1,118.6,-9.0,-8.3,2021-22
26,1610612759,San Antonio Spurs,16,7,9,0.438,768.0,113.3,115.6,111.1,113.3,2.2,2.3,2021-22
27,1610612761,Toronto Raptors,17,11,6,0.647,836.0,111.4,113.7,108.6,110.2,2.8,3.5,2021-22
28,1610612762,Utah Jazz,16,8,8,0.500,768.0,112.4,114.7,107.2,109.8,5.2,4.9,2021-22


In [13]:
last_15.to_csv("last_15.csv", index=False)

### Pull post AS break wins/win% + Save as a csv

In [14]:
### Post all-star break, wins by team, season

datefrom = [''] * 9
columns_post_as = ["TeamID","Team","GP","W","L","WIN%"]

In [16]:
post_as = pull_stats(seasons[0:9], datefrom, 'Base', 'Post+All-Star', 5, columns_post_as)

In [17]:
post_as

,TeamID,Team,GP,W,L,WIN%,Season
0,1610612737,Atlanta Hawks,31,15,16,0.484,2012-13
1,1610612738,Boston Celtics,29,13,16,0.448,2012-13
2,1610612751,Brooklyn Nets,29,18,11,0.621,2012-13
3,1610612766,Charlotte Bobcats,30,9,21,0.300,2012-13
4,1610612741,Chicago Bulls,30,15,15,0.500,2012-13
...,...,...,...,...,...,...,...
25,1610612758,Sacramento Kings,36,17,19,0.472,2020-21
26,1610612759,San Antonio Spurs,40,15,25,0.375,2020-21
27,1610612761,Toronto Raptors,36,10,26,0.278,2020-21
28,1610612762,Utah Jazz,36,25,11,0.694,2020-21


In [18]:
post_as.to_csv("post_as.csv", index=False)